In [1]:
import json
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from PIL import Image
import numpy as np

In [2]:
rawdata = json.load(open('modified_Data_1.json'))["data"]
liwc_ids = np.genfromtxt('liwc_Data.csv', delimiter=',',usecols=(0),dtype=None).tolist()
liwc_data = np.genfromtxt('liwc_Data_noID.csv', delimiter=',')

In [3]:
rawIds = list()
Y = list()
'''just removing "\xef\xbb\xbf" from the first ID'''
liwc_ids[0] = '966590693376781_1502441776458334'

In [4]:
'''there are 3371 rows in raw data and 2971 rows, 93 cols with messages (LIWC)
from raw data, we need dayofweek, month for X and likes for Y'''
picW = 128
picH = 128
numPixels = picW * picH
channels = 3
rawX = np.zeros((len(rawdata), 95 + picW*picH*channels))
print(rawX.shape)

(3371, 49247)


In [5]:
for i in range(len(rawdata)):
    postID = rawdata[i]['id']
    picFileName = rawdata[i]['picFileName']
    rawIds.append(postID)
    rawX[i][0] = rawdata[i]['dayofweek']
    rawX[i][1] = rawdata[i]['month']
    if(postID in liwc_ids) :
        rawX[i][2:95] = liwc_data[liwc_ids.index(postID)]
    if(picFileName != '') :
        img = Image.open('Resized Pics 128x128/'+picFileName).load()
        for imgX in range (picW):
            for imgY in range (picH):
                px = img[imgX, imgY]
                if type(px) is int : r = g = b = px
                elif len(px) is 3 : r, g, b = px
                else : r, g, b, _ = px
                baseIdx = 95 + imgX * picH + imgY
                rawX[i][baseIdx] = r
                rawX[i][baseIdx + numPixels] = g
                rawX[i][baseIdx + numPixels + numPixels] = b
    if(rawdata[i]['likes'] < 100):
        Y.append(0)
    else:
        Y.append(1)

In [6]:
print(len(rawIds))
print(rawX.shape)
print(len(Y))
print(rawX[:,:95].shape)

3371
(3371, 49247)
3371
(3371, 95)


In [7]:
'''split data'''
trainIDs, testIDs, Xtrain_nonimg, Xtest_nonimg, ytrain_nonimg, ytest_nonimg = train_test_split(rawIds, rawX[:,:95], Y[:95], test_size=0.2, random_state=420)

trainIDs, testIDs, Xtrain_img, Xtest_img, ytrain_img, ytest_img = train_test_split(rawIds, rawX[:,95:], Y[:,95:], test_size=0.2, random_state=420)

'''standardize and normalize'''
normalizer = Normalizer()
scaler = StandardScaler()

scaler.fit(Xtrain)
normalizer.fit(Xtrain)

Xtrain = scaler.transform(Xtrain)
Xtrain = normalizer.transform(Xtrain)

Xtest = scaler.transform(Xtest)
Xtest = normalizer.transform(Xtest)

ValueError: Found input variables with inconsistent numbers of samples: [3371, 3371, 95]

In [ ]:
# Create classifiers
clf1 = MLPClassifier(solver='lbfgs', activation='logistic')
clf2 = MLPClassifier(solver='lbfgs', activation='logistic')

In [ ]:
#Fit data
clf1.fit(Xtrain_nonimg, ytrain_nonimg)
clf2.fit(Xtrain_img, ytrain_img)

In [ ]:
#Predict
clf1.predict(Xtest_nonimg)
clf2.predict(Xtest_img)